# HTTP Header Security Check
This notebook checks the security headers of a given URL and allows the user to input cookies and verify a custom CORS setting.

In [4]:

import requests
from colorama import Fore, Style

def check_headers(url, cookies=None, custom_cors=None):
    security_headers = {
        "Strict-Transport-Security": "Missing or misconfigured. Should be set to 'max-age=31536000; includeSubDomains; preload'",
        "Content-Security-Policy": "Missing. Should be set to prevent XSS and other attacks.",
        "X-Frame-Options": "Missing. Should be 'DENY' or 'SAMEORIGIN' to prevent clickjacking.",
        "X-Content-Type-Options": "Missing. Should be 'nosniff' to prevent MIME type sniffing.",
        "Referrer-Policy": "Missing. Should be 'no-referrer' or 'strict-origin-when-cross-origin' to limit referrer information.",
        "Permissions-Policy": "Missing. Should restrict browser features like camera, microphone, and geolocation.",
        "Cache-Control": "Missing. Should be 'no-store' or 'max-age=0, no-cache, must-revalidate' to control caching.",
        "Set-Cookie": "Secure flag missing in cookies. Should have 'Secure; HttpOnly; SameSite=Strict' or 'Lax'.",
        "Access-Control-Allow-Origin": "Missing or misconfigured. Should be set to a specific trusted domain or 'none' to prevent unauthorized cross-origin access."
    }
    
    try:
        headers = {}
        if cookies:
            headers["Cookie"] = cookies
        if custom_cors:
            headers["Origin"] = custom_cors
        
        response = requests.get(url, headers=headers, timeout=10)
        response_headers = response.headers
        
        print(f"Checking security headers for: {url}\n")
        print("Received Headers:")
        for key, value in response_headers.items():
            print(f"{key}: {value}")
        print("\nSecurity Check Results:")
        
        for header, recommendation in security_headers.items():
            if header not in response_headers:
                print(Fore.RED + f"[!] {header} - {recommendation}" + Style.RESET_ALL)
            elif header == "Strict-Transport-Security" and "max-age" not in response_headers[header]:
                print(Fore.RED + f"[!] {header} - Incorrectly configured. {recommendation}" + Style.RESET_ALL)
            elif header == "Set-Cookie" and "Secure" not in response_headers.get("Set-Cookie", ""):
                print(Fore.RED + f"[!] {header} - Secure flag missing in cookies. {recommendation}" + Style.RESET_ALL)
            elif header == "Access-Control-Allow-Origin":
                if custom_cors and response_headers.get(header, "") != custom_cors:
                    print(Fore.RED + f"[!] {header} - Expected '{custom_cors}' but received '{response_headers.get(header, 'None')}'. {recommendation}" + Style.RESET_ALL)
                elif response_headers.get(header, "") == "*":
                    print(Fore.RED + f"[!] {header} - Wildcard '*' is insecure. {recommendation}" + Style.RESET_ALL)
                else:
                    print(Fore.GREEN + f"[+] {header} is properly configured." + Style.RESET_ALL)
            else:
                print(Fore.GREEN + f"[+] {header} is properly configured." + Style.RESET_ALL)
        
    except requests.RequestException as e:
        print(Fore.RED + f"Error fetching {url}: {e}" + Style.RESET_ALL)
    

In [5]:

# User Inputs
target_url = input("Enter the URL to check: ").strip()
cookies = input("Enter cookies (if any, else press Enter to skip): ").strip()
cookies = cookies if cookies else None
custom_cors = input("Enter expected CORS value (if any, else press Enter to skip): ").strip()
custom_cors = custom_cors if custom_cors else None

# Run the security check
check_headers(target_url, cookies, custom_cors)


Checking security headers for: https://www.incognitolab.com

Received Headers:
Content-Type: text/html; charset=utf-8
X-Frame-Options: SAMEORIGIN
X-Content-Type-Options: nosniff
X-XSS-Protection: 1; mode=block
Strict-Transport-Security: max-age=15552000; preload
X-HTML-Minification-Powered-By: WebMarkupMin
X-Akamai-Transformed: 9 - 0 pmb=mRUM,1
Content-Encoding: gzip
Expires: Wed, 05 Mar 2025 12:24:49 GMT
Cache-Control: max-age=0, no-cache, no-store
Pragma: no-cache
Date: Wed, 05 Mar 2025 12:24:49 GMT
Content-Length: 17744
Connection: keep-alive
Vary: Accept-Encoding
Set-Cookie: .AspNetCore.Antiforgery.74e8e92a=CfDJ8MctMbDkPHdFh9z17f3KAoWkMpHPxYCDYqTs8jnWxFL9GjYtrP5d0bAPPc2mtlkqPYghtIIjEXzeG5rA3vv33AVs6r9rTx_YF25bpWO_KdI0vrmcVCG8iDJmqw0YMLQPwZ97PWRc0re2wyBn0IYASRQ; path=/; samesite=strict; httponly
Server-Timing: edge; dur=28, origin; dur=109, cdn-cache; desc=MISS, ak_p; desc="1741177488877_389964262_1862030080_13653_3113_31_44_-";dur=1
Alt-Svc: h3=":443"; ma=93600

Security Check Resu